In [57]:
import pandas as pd
from numpy import poly1d

In [58]:
data = pd.read_csv('data10.csv')

x = data['x']
y = data['y']


Интерполяционный многочлен в форме Ньютона

    Коэффициенты b_n

In [59]:

def f_n(a, x, n):
    return [(a[i + 1] - a[i]) / (x[i + n] - x[i]) for i in range(len(a)-1)]

coefs = [[] for _ in range(len(x))]
coefs[0].extend(f_n(y, x, 1))

for n in range(2, len(coefs)):
    coefs[n - 1].extend(f_n(coefs[n - 2], x, n))

b_n = [y[0]]

for i in range(len(coefs) - 1):
    b_n.append(coefs[i][0])


Коэффициенты b_n

In [60]:
print(*b_n, sep=' ')

0.00123 0.034951011287457756 0.2399694418997176 0.6275508684268281 0.7576347788446149 0.46024316082757594 0.14289126197372365


Коэффициенты интерполяционного многочлена

In [61]:
newton_coefs = [0 for _ in range(len(x))]
newton_coefs[0] += b_n[0]

for i in range(1, len(x)):
    n_poly = poly1d(x[:i], True)
    npc = n_poly.coef    
    k = len(npc) - 1
    j = 0
    while k > -1:
        newton_coefs[k] += b_n[i] * npc[j]
        j += 1
        k -= 1 
print(*newton_coefs, sep=' ')

0.9830666463011832 -4.142168065258056 7.193306859307853 -6.658515905310795 3.51962635242906 -1.0361112767360188 0.14289126197372365


Построение Сплайна 

In [62]:
newton_coefs.reverse()

p1 = poly1d(newton_coefs).deriv(1)

p1_a = [p1(el) for el in x]
spline_coefs = []

def find_spline_coeffs(x, f, P):
    den = (x[1] - x[0]) ** 3

    a_3 = ((P[1] + P[0]) * (x[1] - x[0]) - 2 * (f[1] - f[0])) / den
    a_2 = (3 * (f[1] - f[0]) * (x[1] + x[0]) - (P[1] * (x[1] + 2 * x[0]) + P[0] * (x[0] + 2 * x[1])) * (
            x[1] - x[0])) / den
    a_1 = ((P[1] * x[0] * (2 * x[1] + x[0]) + P[0] * x[1] * (x[1] + 2 * x[0])) * (x[1] - x[0]) - 6 * (f[1] - f[0]) * x[
        0] * x[1]) / den
    a_0 = ((f[1] * x[0] ** 2 * (3 * x[1] - x[0]) + f[0] * x[1] ** 2 * (x[1] - 3 * x[0])) - (
            P[1] * x[0] ** 2 * x[1] + P[0] * x[0] * x[1] ** 2) * (x[1] - x[0])) / den

    return [a_3, a_2, a_1, a_0]

for i in range(len(x) - 1):
    coeffs = find_spline_coeffs([x[i], x[i + 1]], [y[i], y[i + 1]], [p1_a[i], p1_a[i + 1]])
    spline_coefs.append(coeffs)

print(*spline_coefs, sep='\n')



[-0.004320863415796573, 0.15333364296369695, -0.271844216577353, 0.12456018597939383]
[0.5883234456898834, -2.0207523818413127, 2.386662320925761, -0.9590601427064825]
[1.5732464773710992, -6.714942151258398, 9.843294287134414, -4.906810747180746]
[3.6797519303217276, -18.951297579473565, 33.534605094821096, -20.195636516768694]
[7.637130650853382, -46.04299624593536, 95.35466924896546, -67.21615561697305]
[14.174694694835194, -97.59444817993968, 230.8541661572211, -185.9313627520516]


Нахождение значения функции

In [63]:
def F(koord: float, section = x, spline_coefficients = spline_coefs):

    a = 0
    for i in range(len(section) - 1):
        if section[i] - koord <= 1e-10 and 1e-10 <= section[i+1] - koord:
            a = i 
            break
        else:
            a = -1
    print(a)        
    spline_poly = poly1d(spline_coefficients[a])
    print(spline_poly)
    if a == -1:
        return -1
    else:
        return spline_poly(koord)

print(F(1.3))

1
        3         2
0.5883 x - 2.021 x + 2.387 x - 0.9591
0.021075959365861707
